In [71]:
library(hise)
library(ggh4x)
library(ggplot2)
library(stats)
library(parallel)
library(dplyr)
library(readxl)
library(tidyverse)
library(rstatix)
library(ggpubr)
library(corrplot)

In [72]:
meta_data<-read.csv("/home/workspace/IHA_Figure_Revision/Figure4/00_Flu_Year_Specific_Meta_Data/meta_data.csv")


In [73]:
df_MSD<-read.csv("/home/workspace/IHA_Figure_Revision/Figure4/02_Total_IgG_MSD_Assay/MSD_All.csv")
df_MSD<-df_MSD[c("Sample.Kit.Barcode","Assay","Mean","Calc..Conc..Mean","subject.biologicalSex",
             'cohort.cohortGuid', 'subjectGuid','CMV','Flu_Year','Flu_Day','Visit_Type','Year_N','Visit_Flu_Year')]

In [74]:
file_list<-list.files("/home/workspace/IHA_Figure_Revision/Dataset/MSD/")
file_list

[1] "MSD Recent_20231026_092142.csv"             
[2] "Plate 2BLHSAU095_mouse_anti-human_IgG3.xlsx"
[3] "Plate_2BLHSA1081_mouse_anti-human_IgG1.xlsx"
[4] "Plate_2BLHSA4100_mouse_anti-human_IgG4.xlsx"
[5] "Plate_2BLHSAW094_mouse_anti-human_IgG2.xlsx"

In [75]:
df_list<-mclapply(file_list[2:5],function(x){


df<-readxl::read_excel(paste0("/home/workspace/IHA_Figure_Revision/Dataset/MSD/",x), skip = 1) %>% as.data.frame()
df$Isotype<-sub(".*_([^_]+)\\.xlsx$", "\\1", x, perl = TRUE)
df_filtered<- df %>% filter(grepl("PL",Sample)) %>% filter(`Calc. Conc. Mean`!="NaN")
return(df_filtered)
})
df<-do.call(rbind,df_list)

In [76]:
df$sample.sampleKitGuid<-paste0("KT",substr(df$Sample,3,7))

In [77]:
df<-left_join(df,meta_data)

Joining with `by = join_by(sample.sampleKitGuid)`


In [78]:
colnames(df_combined)

[1] "Sample"                  "Assay"                  
 [3] "sample.sampleKitGuid"    "Isotype_Signal"         
 [5] "Signal"                  "log 10 Calc. Conc. Mean"
 [7] "Isotype"                 "Mean"                   
 [9] "Calc..Conc..Mean"        "subject.biologicalSex"  
[11] "cohort.cohortGuid"       "subjectGuid"            
[13] "CMV"                     "Flu_Year"               
[15] "Flu_Day"                 "Visit_Type"             
[17] "Year_N"                  "Visit_Flu_Year"         
[19] "Total_Singal"

In [79]:
df$`Calc.Conc.Mean.Isotype`<-as.numeric(df$`Calc. Conc. Mean`)
df$`log 10 Calc. Conc. Mean`<-log10(as.numeric(df$`Calc. Conc. Mean`))

In [81]:
df_Isotype <- df %>%
    select(Sample, 
           Assay, 
           sample.sampleKitGuid, 
           `Signal`,`log 10 Calc. Conc. Mean`, `Calc.Conc.Mean.Isotype`,
           Isotype)

In [82]:
colnames(df_Isotype)[4]<-"Isotype_Signal"
df_combined <- left_join(df_Isotype, df_MSD, by = c(Assay = "Assay", 
                                                    sample.sampleKitGuid = "Sample.Kit.Barcode"))

In [83]:
df_combined<-df_combined %>% mutate(Total_Singal=Mean)

In [84]:
write.csv(df_combined,"Isotype_MSD_Processed.csv")

# Calculate Ratio

In [16]:
df_combined$Isotype_to_Total_Singal_Ratio<-df_combined$Isotype_Signal/df_combined$Total_Singal

In [17]:
df_combined<-df_combined %>% filter(Isotype_Signal>10,Total_Singal>10)%>% 
filter(Isotype_Signal<1000000,Total_Singal<1000000)

In [20]:
df_combined_subset<-df_combined %>% filter(Isotype %in% c("IgG3","IgG2"), Assay =="Flu B/Phuket HA") 

In [21]:
write.csv(df_combined_subset,"Isotype_Singal_IgG2_IgG3_Phuket.csv")